In [1]:
from lib.model import fromcrn
from get_bifurcations import get_bifurcations
from scipy.interpolate import UnivariateSpline
from numpy import *

from os import system
from pickle import dump,load
from os.path import isfile


from lib.colors import cyan,yellow
from matplotlib.pyplot import *
from numpy import *
from scipy.interpolate import interp2d
from json import loads,dumps

In [2]:
model = fromcrn('double-exclusive.crn')

In [3]:
model.__dict__

{'ATC': 0.0,
 'C12': 0.0,
 'C6': 0.0,
 'IPTG': 0.0,
 'K': 2.7,
 'KGR_76': 0.00145301678863517,
 'KGR_81': 0,
 'KGS_76': 0,
 'KGS_81': 1.00086836995962e-05,
 'KR12': 0,
 'KR6': 3.50695213045775e-08,
 'KS12': 0.0095893786931253,
 'KS6': 0,
 'P76': 0.0,
 'P81': 0.0,
 'a0_76': 1.92,
 'a0_81': 0.822,
 'a1R': 1860.0,
 'a1S': 704,
 'aL': 0.00117,
 'aR33': 9.1,
 'aS175': 3.58,
 'aT': 0.000951,
 'autoC': 3.31,
 'autoY': 0.1,
 'boundLasR': 0.0,
 'boundLuxR': 0.0,
 'boundary': 'nearest',
 'c0': 0.1,
 'c12': 9e-07,
 'c6': 1.8e-06,
 'capacity': 0.0,
 'cb0': 3320,
 'dC12': 0.0,
 'dC6': 0.0,
 'dL': 0.000117,
 'dR': 0.267,
 'dS': 0.319,
 'dT': 0.666,
 'dcfp': 0.129,
 'dimensions': 1,
 'dlasI': 1.09,
 'dluxI': 0.007,
 'dt': 0.01,
 'dyfp': 0.0967,
 'es': 0.5,
 'expressions': {'P76': '(self.a0_76 + self.a1R*self.KGR_76*self.boundLuxR + self.a1S*self.KGS_76*self.boundLasR) / (1.0 + self.KGR_76*self.boundLuxR + self.KGS_76*self.boundLasR)',
  'P81': '(self.a0_81 + self.a1R*self.KGR_81*self.boundLuxR + self

KS12
KS6
c12
c6
lasR
nS


In [66]:
x

'self.lasR**2 * ((self.KS6*self.c6)**self.nS + (self.KS12*self.c12)**self.nS) / ((1.0 + self.KS6*self.c6 + self.KS12*self.c12)**self.nS)'

In [13]:
with open('bifurcations','w') as file :
    dump((c6,c12,L,T),file)

In [43]:
def isfloat(value):
    try:
        float(value)
        return True

    except ValueError:
        return False

In [40]:
def calculate(model,c6,c12,L,T):
    
    Lattractor = array([
        L[argmin(abs(c12[:,0]-c12x)),argmin(abs(c6[0]-c6x))]
        for c6x,c12x in model.state['diffusables'] ]).reshape(-1)

    Tattractor = array([
        T[argmin(abs(c12[:,0]-c12x)),argmin(abs(c6[0]-c6x))]
        for c6x,c12x in model.state['diffusables'] ]).reshape(-1)

    space = linspace(0,model.xmax,len(Lattractor))
    return space,Lattractor,Tattractor

def generate_frame(j,model,space,L,T):
    '''main program figure display'''

    figure(figsize=(10,10))

    # plot scaffolds
    plot(space,10**L,'.',color='darkcyan')
    plot(space,10**T,'.',color='gold')
    plot(-1,-1,'k.',label='Local Steady State')

    # plot system state
    plot(model.space,model.state['inhibitors'].T[0],color='darkcyan')
    plot(model.space,model.state['inhibitors'].T[1],color='gold')
    plot(-1,-1,'k',label='Concentration at $t = {}h$'.format(model.time[-1]))

    legend(fontsize=16)
    xlim(0,model.xmax)
    
    ylabel(r'Inhibitors $L(x,t),T(x,t)$ / nM',fontsize=16);
    xlabel(r'Space $x$ / cm',fontsize=16);
    yscale('log')

    savefig(str(j).zfill(4)+'.png')
    close()

    return j+1
    
def create_animation(C6,C12):
    '''execute system commands to create animation from frames'''
    system('convert -delay 10 -loop 0 *.png c6-{}_c12-{}.gif'.format(C6,C12))
    system('rm *.png')

In [41]:
C6=120;C12=40
t_final = 750.0
p = 0.01

# import model from file
model = fromcrn(crn_path)
model.dt = 0.25

# initial condition
model.state['diffusables'][:,0][model.space<p] = model.xmax * C6 / p
model.state['diffusables'][:,1][model.space>(model.xmax-p)] = model.xmax * C12 / p

# create bifrucation figure
i,j = 0,0
while model.time[-1] < t_final :

    model.time_step()
    space,Lattractor,Tattractor = calculate(model,c6,c12,L,T)
    
    if i % 10 == 0 :
        j = generate_frame(j,model,space,Lattractor,Tattractor)
    i += 1

create_animation(C6,C12)

In [30]:
model.time[-1]

750.0